In [1]:
# This script resfers to https://github.com/evalplus/evalplus
#!pip install evalplus --upgrade

In [2]:
# !pip install transformers
# !pip install bitsandbytes
# !pip install peft
# !pip install accelerate
# !pip install -i https://pypi.org/simple/ bitsandbytes

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, PhiForCausalLM
import torch
from peft import LoraConfig, prepare_model_for_kbit_training, AutoPeftModelForCausalLM, PeftModel

In [4]:
###### TODO: replace model to trained phi-2 #####
torch.set_default_device("cuda")

# model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

In [5]:
from evalplus.data import get_human_eval_plus, write_jsonl
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, PhiForCausalLM
def GEN_SOLUTION(model, problem):
  inputs = tokenizer(problem, return_tensors="pt", return_attention_mask=False).to('cuda')

  outputs = model.generate(**inputs, max_length=len(problem)+300, pad_token_id=tokenizer.eos_token_id)
  text = tokenizer.batch_decode(outputs)[0]

  return text


In [8]:
# Implement the GEN_SOLUTION function by calling the LLM to produce the complete solution (include the code) 
# and save the samples to samples.jsonl:
# Load the model with 4bit quauntization
bnb_config = BitsAndBytesConfig(
        bnb_4bit_compute_dtype='bfloat16',
        bnb_4bit_quant_type='nf4',
        bnb_4bit_use_double_quant=True,
        load_in_4bit=True
)

tokenizer = AutoTokenizer.from_pretrained('trained_models/gemma-2b-it_generation_2', trust_remote_code=True)
tokenizer.padding_side = 'left'
base_model = AutoModelForCausalLM.from_pretrained(
          'google/gemma-2b-it',
          quantization_config=bnb_config,
          trust_remote_code=True
)

# Merge base model with the adapter
model = PeftModel.from_pretrained(base_model, 'trained_models/gemma-2b-it_generation_2')
model = model.merge_and_unload()

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

In [ ]:
samples = [dict(task_id=task_id, solution=GEN_SOLUTION(model, problem["prompt"]))
    for task_id, problem in get_human_eval_plus().items()]
write_jsonl("samples.jsonl", samples)  ### write the generate response to local

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/ace/.virtualenvs/545project/lib/python3.11/site-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [9]:
!git clone https://github.com/evalplus/evalplus.git

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Cloning into 'evalplus'...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


remote: Enumerating objects: 6405, done.
remote: Counting objects: 100% (6402/6402), done.
remote: Compressing objects: 100% (2257/2257), done.
remote: Total 6405 (delta 4011), reused 6146 (delta 3900), pack-reused 3
Receiving objects: 100% (6405/6405), 5.20 MiB | 8.82 MiB/s, done.
Resolving deltas: 100% (4011/4011), done.


In [10]:
!cd ./evalplus/evalplus

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
# LLM-generated text may not be compilable code for including natural language lines or incomplete extra code.
# We provide a tool namely evalplus.sanitize to clean up the code:
!python ./evalplus/sanitize.py --samples ./samples.jsonl

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: line 1: python: command not found


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
# Checking the compilability of post-processed code:
!python evalplus/syncheck.py --samples ./samples-sanitized.jsonl --dataset humaneval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: line 1: python: command not found
